In [84]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import networkx as nx
import csv
seed = 123234345

def create_fully_random_graph(number_of_nodes, Average_degree):
    #create random list with mean = Average_degree
    Total = number_of_nodes*Average_degree
    Upper_boundary = 100

    probs = np.full(number_of_nodes, 1.0/np.float64(number_of_nodes), dtype=np.float64) # probabilities

    N = 10000 # samples to test
    for k in range(N):
        table_of_edges = np.random.multinomial(Total, probs)
        t = np.where(table_of_edges > Upper_boundary) # check for out-of boundaries
        if np.any(t):
            continue
 
    G = nx.random_degree_sequence_graph(table_of_edges) #Graph initialization
    
    return G

def create_random_graph_with_precise_n_egdees(number_of_nodes, Average_degree):
    G = nx.random_regular_graph(Average_degree, number_of_nodes, seed) #Graph initialization
    
    return G

def mean_n_edges(Graph):
    list_edges = []
    list = [v for v in Graph.degree()]
    for i in range(0, len(list)):
        list_edges.append(list[i][1])
    
    return np.mean(list_edges)

def define_env(number_of_nodes, Average_degree):
    G = create_random_graph_with_precise_n_egdees(number_of_nodes, Average_degree)
    #G = create_fully_random_graph(number_of_nodes, Average_degree)
    
    return G

In [85]:
def set_state(G):
    init_states = [1 for _ in range(number_of_nodes)]
    for i in range(number_of_nodes):
        if random.random() < 0.5:
            init_states[i] = -1
    
    states = {new_list: [] for new_list in range(number_of_nodes)} #set_node_attributes works only with dictionary
    for i in range(number_of_nodes):
        states[i].append(init_states[i])
        
    nx.set_node_attributes(G, states, 'states') #set node states by states value
        
def convince(node_id, G):
        #print("teraz pracuje na ty nodzie: ", node_id)
        #print("taki ma stan: ", G.nodes[node_id]['states'][0])
        #print("takich ma sasiadow: ", [n for n in G.neighbors(node_id)])
        node_neighbors = [n for n in G.neighbors(node_id)] #making list of all node neighbors
        random.shuffle(node_neighbors) #shuffle this list randomly
        for neighbor in node_neighbors: #for all node neighbors
            if G.nodes[neighbor]['states'][0] != G.nodes[node_id]['states'][0]: #if states of node and neighbor are different
                
                #print("upewniam się że mają inne stany ", G.nodes[node_id]['states'][0], G.nodes[neighbor]['states'][0])
                #print("to ten sąsiad: ",neighbor)

                if random.random() < (1-convince_prob):  #change node stat with prob = 1-convince_prob           
                    G.nodes[node_id]['states'][0] = G.nodes[neighbor]['states'][0]
                    #print("teraz mają takie same stany: ", G.nodes[node_id]['states'][0], G.nodes[neighbor]['states'][0])
                    
                else: # or with prob = convince_prob 
                    G.remove_edge(node_id, neighbor) #remove enge
                    for every_node in range(number_of_nodes): #for number of nodes in graph
                        new_edge = random.randrange(number_of_nodes) #take random node
                        if new_edge not in G.neighbors(node_id): #if this new node is not our node neighbor
                            #print("oto nowe połączenie: ",new_edgee)
                            G.add_edge(node_id, new_edge) #connect this two nodes
                            break #we want only 1 new edge
                            
                #print("teraz takich ma sasiadow: ", [n for n in G.neighbors(node_id)])
            break

def take_egdee_list(G):
    positive = 0
    negative = 0
    for node in G.nodes: #search all nodes
        if G.nodes[node]['states'][0] == 1: #when node has state = 1
            for n in G.neighbors(node):  #search all node neighbors
                if G.nodes[n]['states'][0] == 1: #if node neighbor state is 1 add 1 to positive variable
                    #print("+", node, n)
                    positive += 1
                        
        if G.nodes[node]['states'][0] == -1: #do the same for -1 state
            for n in G.neighbors(node):
                if G.nodes[n]['states'][0] == -1:
                    #print("-", node, n)
                    negative += 1
    tab = []             
    tab.append(positive)
    tab.append(negative)
    return tab

def take_number_of_actife_links(G): #adding 1 to 'active' variable, when states of nodes are different
    active = 0
    for node in G.nodes: 
        for neighbor in G.neighbors(node):
            if G.nodes[node]['states'][0] != G.nodes[neighbor]['states'][0]:
                active += 1

    return active
            
def run(G):
    node_id = random.randrange(number_of_nodes) #set node id randomly
    convince(node_id, G) #heart of simulation
    #make_plot(G) #if tou want to make sim visualization
        
def run_simulation(number_of_nodes, Average_degree, iterations):
    G = define_env(number_of_nodes, Average_degree) #define graph
    set_state(G) #define states of nodes
    
    for i in range(iterations): #run sim i-times
        run(G)
    
    links = take_egdee_list(G) #take list of edges
    #print("positive links: ", links[0], "negative links: ", links[1])
    #print("active links: ", take_number_of_actife_links(G))
    #print("number of connected components: ", len(max(nx.connected_components(G), key = len)))
    
    data = {'active_links': [take_number_of_actife_links(G)],'n_of_c_comp': [len(max(nx.connected_components(G), key = len))],
            'p_links': [links[0]], 'n_links': [links[1]]}
    df = pd.DataFrame(data, columns = ["active_links", "n_of_c_comp", "p_links", "n_links"])
    
    #make_plot(G)
    return df
    

def measure_state_stan_time(number_of_nodes, Average_degree):
    G = define_env(number_of_nodes, Average_degree) #define graph
    set_state(G) #define states of nodes
    counter = 0 #define counter
    active = take_number_of_actife_links(G) #define number of active links
    
    while active!= 0: #loop will end when number of active links = 0
        run(G)
        active = take_number_of_actife_links(G)
        if counter % 1000 == 0:
            print(counter)
            #print(active)
        counter += 1
    
    print("counter: ", counter)
    
    return counter

In [86]:
def make_plot_kamada_kawai(G):
    node_color = []
    pos = nx.kamada_kawai_layout(G, scale = 1)
    for node in G.nodes(data=True):
        # if the node has the attribute 1
        if 1 in node[1]['states']:
            node_color.append('red')
        elif -1 in node[1]['states']:
            node_color.append('blue')
        
    nx.draw(G, pos, with_labels=True, font_weight='normal', node_size= 200, node_color=node_color) #plot declaration
    plt.figure(figsize=(100,100))
    plt.show()
    
def make_plot(G):
    node_color = []
    for node in G.nodes(data=True):
        # if the node has the attribute 1
        if 1 in node[1]['states']:
            node_color.append('red')
        elif -1 in node[1]['states']:
            node_color.append('blue')
            
    nx.draw(G, with_labels=True, font_weight='normal', node_size= 200, node_color=node_color)
    plt.figure(figsize=(100,100))
    plt.draw()

In [87]:
def run_series_of_sim(noumber_of_sim, number_of_nodes, Average_degree, iterations, file_name):
    print('Starting simulations...')
    with open(file_name, 'a') as f:
        for i in range(noumber_of_sim):
            df = run_simulation(number_of_nodes, Average_degree, iterations)
            df.to_csv(f, index = False, header=False, line_terminator="\n")
            if i%10 == 0:
                print("progres: ", i,"/", noumber_of_sim)
    print("progres: ", noumber_of_sim,"/", noumber_of_sim)
    print('Simulation completed.')
        
def run_series_of_measure_time(noumber_of_sim, number_of_nodes, Average_degree, file_name):
    print('Starting simulations...')
    with open(file_name, 'a') as f:
        for i in range(noumber_of_sim):
            counter = measure_state_stan_time(number_of_nodes, Average_degree)
            f.write(str(counter))
            f.write("\n")            
    print('Simulation completed.')

In [88]:
#properties declaration
convince_prob = 0.5
number_of_nodes = 250
Average_degree = 4

#run_series_of_sim(number of simulations, number of nodes, average degree, number of iterations in one simulation, "filename.csv")
run_series_of_sim(100, number_of_nodes, Average_degree, 10000, "plik.csv")

#run_series_of_measure_time(number of simulations, number of nodes, average degree, number of iterations in one simulation, "filename.csv")
#run_series_of_measure_time(2, number_of_nodes, Average_degree, "plik2.csv")

Starting simulations...
progres:  0 / 100
progres:  10 / 100
progres:  20 / 100
progres:  30 / 100
progres:  40 / 100
progres:  50 / 100
progres:  60 / 100
progres:  70 / 100
progres:  80 / 100
progres:  90 / 100
progres:  100 / 100
Simulation completed.
